In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
git clone https://github.com/huggingface/neuralcoref.git

Cloning into 'neuralcoref'...
Checking out files: 100% (151/151), done.


In [ ]:
# !git clone https://github.com/huggingface/neuralcoref.git
!pip install https://huggingface.co/spacy/en_core_web_sm/resolve/main/en_core_web_sm-any-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 27.3 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.1
    Uninstalling en-core-web-sm-3.4.1:
      Successfully uninstalled en-core-web-sm-3.4.1


In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd neuralcoref

!pip install -r requirements.txt
!pip install -e .

/content/neuralcoref
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.0 MB 22.2 MB/s 
     |████████████████████████████████| 1.1 MB 36.6 MB/s 
     |████████████████████████████████| 211 kB 46.1 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.3
    Uninstalling spacy-3.4.3:
      Successfully uninstalled spacy-3.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [ ]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 19.6 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047104 sha256=a15ddbda3003a85c015139a24ad7cdcdf1ce88ddeda0f723bb0241b9b6945787
  Stored in directory: /root/.cache/pip/wheels/ee/4d/f7/563214122be1540b5f9197b52cb3ddb9c4a8070808b22d5a84
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.0
    Uninstalling en-core-web-sm-3.4.0:
      Successfully uninstalled en-core-web-sm-3.4.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

## Downloading the data

We'll be working with an English-to-Spanish translation dataset
provided by [Anki](https://www.manythings.org/anki/). Let's download it:

In [ ]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

2638744/2638744 [==============================] - 0s 0us/step


In [ ]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en_core_web_sm')

# load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

# You're done. You can now use NeuralCoref the same way you usually manipulate a SpaCy document and it's annotations.

100%|██████████| 40155833/40155833 [00:00<00:00, 64232915.85B/s]


## Load in Books Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
books_english_path = "/content/drive/MyDrive/6.8610 [6.864] Quant Methods for NLP/Project/book-en-sentences.txt"
books_spanish_path = "/content/drive/MyDrive/6.8610 [6.864] Quant Methods for NLP/Project/book-es-sentences.txt"

with open(books_english_path) as engBooks:
    english_lines = engBooks.read().split("\n")[:-1]

with open(books_spanish_path) as spaBooks:
    spanish_lines = spaBooks.read().split("\n")[:-1]

## Parsing the data

Each line contains an English sentence and its corresponding Spanish sentence.
The English sentence is the *source sequence* and Spanish one is the *target sequence*.
We prepend the token `"[start]"` and we append the token `"[end]"` to the Spanish sentence.

In [ ]:
feminine_markers = {'Ella', 'ella', 'Ellas', 'ellas'}
feminine_names = {'Mary', 'mary', "Elinor", "Marianne", 'Margaret', 'Fanny', 'Karenina', 'Anna', 'Kitty', 'Eliza', 'Catherine', 'Ms.', 'Mrs.', 'Beryl', 'Cunegonde', 'Miss'}
masculine_markers = {'Él', 'él', 'Ellos', 'ellos'}
masculine_names = {'Tom', 'tom', 'Leo', 'leo'}

### Parse Books Dataset

Note: this dataset has a book in French. The book starts at the phrase: "Un crime étrange", and ends at the phrase: "The Hound of the Baskervilles". This book will be filtered out.

In [ ]:
book_text_pairs = []

feminine_count = 0

found_french = False

for i in range(len(english_lines)):
  english_phrase = english_lines[i]
  spanish_phrase = spanish_lines[i]

  if english_phrase == "Un crime étrange":
    print("Setting to True, found French")
    found_french = True

  if english_phrase == "The Hound of the Baskervilles":
    print("setting to False, found Hound")
    found_french = False
  
  if found_french == True :
    continue

  # Include sentence if contains female pronoun
  spa_set = set(spanish_phrase.split())
  eng_set = set(english_phrase.split())

  if feminine_markers.intersection(spa_set) or feminine_names.intersection(eng_set):
    feminine_count += 1
    doc = nlp(english_phrase)
    engList = english_phrase.split()
    coreferences = doc._.coref_clusters

    newEng = english_phrase

    corefLookup = {} # all objects -> original subject

    for coref in coreferences:
      corefSubject, corefObjects = coref.mentions[0], set(coref.mentions[1:])

      for obj in corefObjects:
        corefLookup[obj] = corefSubject
        newEng = newEng.replace(" " + str(obj) + " ", " [" + str(corefSubject) + "] " + str(obj) + " ")
        newEng = newEng.replace(" " + str(obj) + ".", " [" + str(corefSubject) + "] " + str(obj) + ".")
    
    adjEng = newEng
    for word in doc:
      # check if noun or "is" / "are" / "was" / "were" 
      auxWords = {"is", "are", "was", "were"}
      adjAlreadyAdded = set()
      if word.pos_ == "NOUN" or word.pos_ == "PRON":
        children = [child for child in word.children]

        for child in children:
          if child.pos_ == "ADJ" and child.text not in adjAlreadyAdded:

            adjAlreadyAdded.add(child.text)
            adjObject = corefLookup[word.text] if word.text in corefLookup else word.text

            adjEng = adjEng.replace(" " + str(child.text) + " ", " [" + str(adjObject) + "] " + str(child.text) + " ")
            adjEng = adjEng.replace(" " + str(child.text) + ".", " [" + str(adjObject) + "] " + str(child.text) + ".")

      elif word.text in auxWords:
        children = [child for child in word.children]
        
        adjective = []
        nounPronoun = []

        for child in children:
          if child.pos_ == "ADJ":
            adjective.append(child)
          if child.pos_ == "NOUN" or child.pos_ == "PRON":
            nounPronoun.append(child)

        if adjective and nounPronoun and adjective[0].text not in adjAlreadyAdded:
          adjAlreadyAdded.add(adjective[0].text)
          adjObject = corefLookup[nounPronoun[0].text] if nounPronoun[0].text in corefLookup else nounPronoun[0].text
          adjEng = adjEng.replace(" " + str(adjective[0].text) + " ", " [" + str(adjObject) + "] " + str(adjective[0].text) + " ")
          adjEng = adjEng.replace(" " + str(adjective[0].text) + ".", " [" + str(adjObject) + "] " + str(adjective[0].text) + ".")
    
    english_phrase = adjEng

    spanish_phrase = "[start] " + spanish_phrase + " [end]"
    book_text_pairs.append((english_phrase, spanish_phrase))

    if feminine_count % 10000 == 0:
      print("Processed: ", feminine_count)

Setting to True, found French
setting to False, found Hound


In [ ]:
print(feminine_count)

print(len(book_text_pairs))

5079
5079


In [ ]:
print(feminine_count)

print(len(book_text_pairs))

4582
4582


In [ ]:
print(feminine_count)

print(len(book_text_pairs))

2568
2568


### Parse Other Dataset

In [ ]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

In [ ]:
text_pairs = []

counter = 0
male_count = 0
male_added = 0
ungendered = 0
female_count = 0

for line in lines:
# for i in range(len(lines) - 15, len(lines)):
    eng, spa = line.split("\t")
    spa_set = set(spa.split())
    eng_set = set(eng.split())

    addLine = False

    if feminine_markers.intersection(spa_set) or feminine_names.intersection(eng_set) or feminine_names.intersection(spa_set):
      addLine = True
      female_count += 1
    elif masculine_markers.intersection(spa_set) or masculine_names.intersection(eng_set) or masculine_names.intersection(spa_set):
      if male_count % 2 == 0:
        male_added += 1
        addLine = True
      male_count += 1
    else:
      addLine = True
      ungendered += 1

    if addLine:
      doc = nlp(eng)
      engList = eng.split()
      coreferences = doc._.coref_clusters

      newEng = eng

      corefLookup = {} # all objects -> original subject

      for coref in coreferences:
        corefSubject, corefObjects = coref.mentions[0], set(coref.mentions[1:])

        for obj in corefObjects:
          corefLookup[obj] = corefSubject
          newEng = newEng.replace(" " + str(obj) + " ", " [" + str(corefSubject) + "] " + str(obj) + " ")
          newEng = newEng.replace(" " + str(obj) + ".", " [" + str(corefSubject) + "] " + str(obj) + ".")
      
      adjEng = newEng
      for word in doc:
        # check if noun or "is" / "are" / "was" / "were" 
        auxWords = {"is", "are", "was", "were"}
        adjAlreadyAdded = set()
        if word.pos_ == "NOUN" or word.pos_ == "PRON":
          children = [child for child in word.children]

          for child in children:
            if child.pos_ == "ADJ" and child.text not in adjAlreadyAdded:

              adjAlreadyAdded.add(child.text)
              adjObject = corefLookup[word.text] if word.text in corefLookup else word.text

              adjEng = adjEng.replace(" " + str(child.text) + " ", " [" + str(adjObject) + "] " + str(child.text) + " ")
              adjEng = adjEng.replace(" " + str(child.text) + ".", " [" + str(adjObject) + "] " + str(child.text) + ".")

        elif word.text in auxWords:
          children = [child for child in word.children]
          
          adjective = []
          nounPronoun = []

          for child in children:
            if child.pos_ == "ADJ":
              adjective.append(child)
            if child.pos_ == "NOUN" or child.pos_ == "PRON":
              nounPronoun.append(child)

          if adjective and nounPronoun and adjective[0].text not in adjAlreadyAdded:
            adjAlreadyAdded.add(adjective[0].text)
            adjObject = corefLookup[nounPronoun[0].text] if nounPronoun[0].text in corefLookup else nounPronoun[0].text
            adjEng = adjEng.replace(" " + str(adjective[0].text) + " ", " [" + str(adjObject) + "] " + str(adjective[0].text) + " ")
            adjEng = adjEng.replace(" " + str(adjective[0].text) + ".", " [" + str(adjObject) + "] " + str(adjective[0].text) + ".")
      
      eng = adjEng

      spa = "[start] " + spa + " [end]"
      text_pairs.append((eng, spa))
      counter += 1

      if counter % 10000 == 0:
        print("Processed: ", counter)

Processed:  10000
Processed:  20000
Processed:  30000
Processed:  40000
Processed:  50000
Processed:  60000
Processed:  70000
Processed:  80000
Processed:  90000
Processed:  100000


In [ ]:
print(len(text_pairs))

print("Male_total: ", male_count)

print("Male_added: ", male_added)

print("Female_total: ", female_count+ feminine_count)

print("Ungendered: ", ungendered)


107623
Male_total:  22683
Male_added:  11342
Female_total:  12609
Ungendered:  88751


In [ ]:
text_pairs = text_pairs + book_text_pairs

In [ ]:
print(len(text_pairs))

print("Male_total: ", male_count)

print("Male_added: ", male_added)

print("Female_total: ", female_count+ feminine_count)

print("Ungendered: ", ungendered)

112702
Male_total:  22683
Male_added:  11342
Female_total:  12609
Ungendered:  88751


Here's what our sentence pairs look like:

In [ ]:
for _ in range(1000):
    print(random.choice(text_pairs))

('I drank from the tap.', '[start] Tomé del grifo. [end]')
("I can't reach that can of tomatoes.", '[start] No puedo alcanzar esa lata de tomate. [end]')
('My [brother] younger brother is watching TV.', '[start] Mi hermano pequeño está viendo la televisión. [end]')
("Three-fourths of the earth's surface is covered with water.", '[start] Tres cuartos de la superficie de la Tierra están cubiertos de agua. [end]')
("If you don't eat, you'll die.", '[start] Si no comes, te mueres. [end]')
("It's your money, isn't it?", '[start] Ese dinero es tuyo, ¿verdad? [end]')
('Have you worked out the answer yet?', '[start] ¿Has encontrado la respuesta? [end]')
("I'm sorry, Tom, I've got to go.", '[start] Lo siento Tom, me tengo que ir. [end]')
('Those are your options.', '[start] Esas son tus opciones. [end]')
('You have been warned.', '[start] Has sido advertido. [end]')
("Don't make me laugh.", '[start] No me hagas reír. [end]')
("I don't want to leave you.", '[start] No quiero dejarte. [end]')
('M

Now, let's split the sentence pairs into a training set, a validation set,
and a test set.

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

112702 total pairs
78892 training pairs
16905 validation pairs
16905 test pairs


## Vectorizing the text data

We'll use two instances of the `TextVectorization` layer to vectorize the text
data (one for English and one for Spanish),
that is to say, to turn the original strings into integer sequences
where each integer represents the index of a word in a vocabulary.

The English layer will use the default string standardization (strip punctuation characters)
and splitting scheme (split on whitespace), while
the Spanish layer will use a custom standardization, where we add the character
`"¿"` to the set of punctuation characters to be stripped.

Note: in a production-grade machine translation model, I would not recommend
stripping the punctuation characters in either language. Instead, I would recommend turning
each punctuation character into its own token,
which you could achieve by providing a custom `split` function to the `TextVectorization` layer.

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the vectorized source sentence and `encoder_inputs` is the target sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

In [ ]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 20 steps long):

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


## Building the model

Our sequence-to-sequence Transformer consists of a `TransformerEncoder`
and a `TransformerDecoder` chained together. To make the model aware of word order,
we also use a `PositionalEmbedding` layer.

The source sequence will be pass to the `TransformerEncoder`,
which will produce a new representation of it.
This new representation will then be passed
to the `TransformerDecoder`, together with the target sequence so far (target words 0 to N).
The `TransformerDecoder` will then seek to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking
(see method `get_causal_attention_mask()` on the `TransformerDecoder`).
The `TransformerDecoder` sees the entire sequences at once, and thus we must make
sure that it only uses information from target tokens 0 to N when predicting token N+1
(otherwise, it could use information from the future, which would
result in a model that cannot be used at inference time).


In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config

In [ ]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
epochs = 5  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

In [ ]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(200):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, translated)

It's just what we need. [start] es lo que necesitamos [end]
Do you have two books? [start] tiene dos libros [end]
I will be back soon. [start] yo será pronto [end]
I'll pick you up at your home at five. [start] te [UNK] en su casa a las cinco [end]
I thought I might be able to help. [start] pensé que yo podría ayudar [end]
"Bah!" said Milady, with a smile that belonged only to herself; "you forget that I am [Milady] her [friend] best friend." [start] [UNK] dijo que le ha dicho que ella no le [UNK] que yo era una buena [UNK] [end]
The car exploded a [moments] few moments after the collision. [start] el auto se dio unos minutos después de la cena [end]
It was night. [start] fui anoche [end]
He has four [phones] mobile phones. [start] Él tiene cuatro [UNK] [end]
I'm confident that I'll pass the exam. [start] estoy [UNK] así que yo [UNK] el examen [end]
I'm psyched. [start] estoy [UNK] [end]
When did you come back from Boston? [start] cuándo vino aquí de boston [end]
That's wonderful! [sta

In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
translated_sentences = []
for i in range(400):
  input_sentence = test_eng_texts[i]
  translated = decode_sequence(input_sentence)
  translated_sentences.append(translated)

test_sp_texts = [pair[1] for pair in test_pairs]

bleu_scores = []
for j in range(len(translated_sentences)):
  bleu_scores.append(sentence_bleu(translated_sentences[j], test_sp_texts[j], weights = (1, 0, 0, 0)))
  print(translated_sentences[j], test_sp_texts[j] )
print(bleu_scores) 
print(np.sum(np.array(bleu_scores))/len(bleu_scores))

[start] no puedo hablar inglés tan bien como el inglés [end] [start] No hablo inglés tan bien como él. [end]
[start] sé por qué lo [UNK] [end] [start] Yo sé por qué lo necesitas. [end]
[start] no hay ningún cuarto en la habitación [end] [start] No hay mesas en la habitación. [end]
[start] quiero ser rico [end] [start] Quiero ser rica. [end]
[start] cuando el tren va a la estación de [UNK] y [UNK] de la casa de la casa [end] [start] Cuando el tren pasa por la estación, las ventanas suenan fuerte y toda la casa tiembla. [end]
[start] si estás listo el trabajo [end] [start] Si estás listo, vamos a empezar. [end]
[start] el brazo ha estado todavía [end] [start] Mi brazo izquierdo está dormido. [end]
[start] el teléfono se [UNK] [end] [start] El teléfono sonó repetidamente. [end]
[start] ellos [UNK] el primer primer [UNK] y [UNK] [end] [start] Ellos combatieron a los italianos primero en Egipto y Libia. [end]
[start] [UNK] [end] [start] Ayúdalo. [end]
[start] este es para mi amigo [end] [st